Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import catboost as cb
from datetime import datetime
import seaborn as sns

In [2]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")
data_webstat = pd.read_csv("./data/t1_webstat.csv")

is_callcenter - был ли совершен звонок в коллцентр по итогам сессии (нужно предсказать)

order_id - id заказа

create_time - время создания заказа

good_id - id товара

price - цена товара

utm_medium - тип трафика

utm_source - источник трафика

sessionkey_id - id сессии пользователя

category_id - id категории товара

parent_id - id родительской категории для категории товара

root_id - id "базовой" категории товара

rating_value - рейтинг товара

rating_count - число оценок товара

description_length - длина описания товара

goods_qty - число товаров в заказе

model_id - ???

is_moderated - ???

pics_qty - ???

model_create_time - ???

sessionkey_id - id сессии пользователя

date_time - timestamp захода на страницу определенного типа

page_type - тип просмотренной страницы

pageview_number - порядковый номер страницы среди всех страниц, отсортированных по времени посещения внутри сессии

pageview_duration_sec - время просмотра страницы

product_in_sale - ???

In [3]:
data_train["create_time_ts"] = None
data_test["create_time_ts"] = None
data_train["model_create_time_ts"] = None
data_test["model_create_time_ts"] = None

from datetime import datetime
# convert string to date
data_train.create_time_ts = data_train.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.create_time_ts = data_test.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

data_train.loc[~data_train.model_create_time.isna(), "model_create_time_ts"] = data_train.loc[~data_train.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.loc[~data_test.model_create_time.isna(), "model_create_time_ts"] = data_test.loc[~data_test.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

In [4]:
del data_train["create_time"]
del data_train["model_create_time"]
del data_test["create_time"]
del data_test["model_create_time"]

In [5]:
data_train.model_create_time_ts = data_train.model_create_time_ts.fillna(data_train.create_time_ts)
data_test.model_create_time_ts = data_test.model_create_time_ts.fillna(data_test.create_time_ts)

In [123]:
print("Рейтинг товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["rating_value"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["rating_value"].mean())

Рейтинг товаров, по которым позвонили:  1.5399876007439555 , по которым не звонили 1.4771690174232548


In [124]:
print("Длина описания товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["description_length"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["description_length"].mean())

Длина описания товаров, по которым позвонили:  572.92983638373 , по которым не звонили 573.338671921299


In [125]:
print("Представителей 0 класса: 67496, 1 класса: 37099")

Представителей 0 класса: 67496, 1 класса: 37099


In [15]:
A = data_train.groupby(['root_id'])['is_callcenter'].sum().sort_values()
B = data_train.groupby(['root_id'])['is_callcenter'].count().sort_values()
C = A.divide(B).sort_values()

order_id = list(data_train["order_id"].unique())
new_column = list()
for order in order_id:
    root = list(data_train[data_train["order_id"] == order]["root_id"].values)[0]
    new_column.append(C[root])
new_column_test = list()
for order in list(data_test["order_id"].unique()):
    root = list(data_test[data_test["order_id"] == order]["root_id"].values)[0]
    new_column_test.append(C[root])
data_train['root_mean_call'] = new_column
data_test['root_mean_call'] = new_column_test

In [27]:
new_column = list()
for order in range(data_train.shape[0]):
    hour = data_train["create_time_ts"][order].time().hour
    new_column.append(hour)
data_train['hour'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    hour = data_test["create_time_ts"][order].time().hour
    new_column.append(hour)
data_test['hour'] = new_column

In [54]:
data_train["utm_source"].fillna(0, inplace=True)
data_test["utm_source"].fillna(0, inplace=True)

In [66]:
data_train["rating_value"].fillna(0, inplace=True)
data_test["rating_value"].fillna(0, inplace=True)

In [75]:
data_train["rating_count"].fillna(0, inplace=True)
data_test["rating_count"].fillna(0, inplace=True)

In [149]:
new_column = list()
for order in range(data_train.shape[0]):
    day = data_train["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_train['day_of_week'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    day = data_test["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_test['day_of_week'] = new_column

In [164]:
data_train["life_time"] = (data_train.create_time_ts - data_train.model_create_time_ts).astype('timedelta64[D]')
data_test["life_time"] = (data_test.create_time_ts - data_test.model_create_time_ts).astype('timedelta64[D]')

In [166]:
features = ["pics_qty", 
            "goods_qty", 
            "description_length", 
            "is_moderated", 
            "model_id", 
            "parent_id", 
            "category_id", 
            "utm_medium", 
            "price", 
            "root_mean_call",
            "hour",
            "utm_source",
            "rating_value",
            "day_of_week",
            "life_time"]

# ОБУЧЕНИЕ (TEST)

In [167]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import cv
from catboost import Pool

In [168]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.15, random_state=42, stratify=data_train["is_callcenter"])
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42, stratify = y_train_val)

In [169]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)

In [170]:
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.092227
0:	test: 0.6155362	best: 0.6155362 (0)	total: 13.7ms	remaining: 13.7s
100:	test: 0.6680572	best: 0.6680572 (100)	total: 1.04s	remaining: 9.28s
200:	test: 0.6697858	best: 0.6700055 (187)	total: 2.01s	remaining: 8.01s
300:	test: 0.6702379	best: 0.6703312 (215)	total: 2.98s	remaining: 6.91s
400:	test: 0.6702289	best: 0.6709036 (367)	total: 3.92s	remaining: 5.86s
500:	test: 0.6708563	best: 0.6710083 (482)	total: 4.87s	remaining: 4.85s
600:	test: 0.6708075	best: 0.6711741 (513)	total: 5.89s	remaining: 3.91s
700:	test: 0.6709149	best: 0.6711741 (513)	total: 6.82s	remaining: 2.91s
800:	test: 0.6701187	best: 0.6711741 (513)	total: 7.78s	remaining: 1.93s
900:	test: 0.6699177	best: 0.6711741 (513)	total: 8.72s	remaining: 959ms
999:	test: 0.6694216	best: 0.6711741 (513)	total: 9.67s	remaining: 0us

bestTest = 0.6711740518
bestIteration = 513

Shrink model to first 514 iterations.


In [171]:
predictions = model.predict(X_test[features])

In [172]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

0.6243378256963164

# ОБУЧЕНИЕ (Kaggle)

In [173]:
X_train, X_val, y_train, y_val = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.2, random_state=42, stratify=data_train["is_callcenter"])

In [174]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.094578
0:	test: 0.6262880	best: 0.6262880 (0)	total: 17.7ms	remaining: 17.7s
100:	test: 0.6669814	best: 0.6670423 (99)	total: 1.13s	remaining: 10.1s
200:	test: 0.6691259	best: 0.6691259 (200)	total: 2.2s	remaining: 8.75s
300:	test: 0.6703711	best: 0.6707947 (270)	total: 3.23s	remaining: 7.51s
400:	test: 0.6708051	best: 0.6710889 (382)	total: 4.26s	remaining: 6.36s
500:	test: 0.6699396	best: 0.6710889 (382)	total: 5.29s	remaining: 5.27s
600:	test: 0.6693942	best: 0.6710889 (382)	total: 6.31s	remaining: 4.19s
700:	test: 0.6689732	best: 0.6710889 (382)	total: 7.36s	remaining: 3.14s
800:	test: 0.6688600	best: 0.6710889 (382)	total: 8.4s	remaining: 2.09s
900:	test: 0.6680616	best: 0.6710889 (382)	total: 9.44s	remaining: 1.04s
999:	test: 0.6672579	best: 0.6710889 (382)	total: 10.5s	remaining: 0us

bestTest = 0.6710888737
bestIteration = 382

Shrink model to first 383 iterations.


In [175]:
predictions = model.predict(data_test[features])

In [176]:
id_column = data_test["order_id"]
result_column = predictions
dataset = pd.DataFrame({'order_id': id_column, 'is_callcenter': list(result_column)}, columns=['order_id', 'is_callcenter'])
dataset.to_csv("sample_submission.csv", index=False)